In [1]:
from pathlib import Path

import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from tqdm import tqdm

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
seed = 42
torch.manual_seed(seed)

CLASSES = [
    "N/A",
    "person",
    "bicycle",
    "car",
    "motorcycle",
    "airplane",
    "bus",
    "train",
    "truck",
    "boat",
    "traffic light",
    "fire hydrant",
    "N/A",
    "stop sign",
    "parking meter",
    "bench",
    "bird",
    "cat",
    "dog",
    "horse",
    "sheep",
    "cow",
    "elephant",
    "bear",
    "zebra",
    "giraffe",
    "N/A",
    "backpack",
    "umbrella",
    "N/A",
    "N/A",
    "handbag",
    "tie",
    "suitcase",
    "frisbee",
    "skis",
    "snowboard",
    "sports ball",
    "kite",
    "baseball bat",
    "baseball glove",
    "skateboard",
    "surfboard",
    "tennis racket",
    "bottle",
    "N/A",
    "wine glass",
    "cup",
    "fork",
    "knife",
    "spoon",
    "bowl",
    "banana",
    "apple",
    "sandwich",
    "orange",
    "broccoli",
    "carrot",
    "hot dog",
    "pizza",
    "donut",
    "cake",
    "chair",
    "couch",
    "potted plant",
    "bed",
    "N/A",
    "dining table",
    "N/A",
    "N/A",
    "toilet",
    "N/A",
    "tv",
    "laptop",
    "mouse",
    "remote",
    "keyboard",
    "cell phone",
    "microwave",
    "oven",
    "toaster",
    "sink",
    "refrigerator",
    "N/A",
    "book",
    "clock",
    "vase",
    "scissors",
    "teddy bear",
    "hair drier",
    "toothbrush",
]

2022-11-23 19:26:31.149028: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-23 19:26:31.683054: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/matias/RECVIS:
2022-11-23 19:26:31.683082: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-23 19:26:31.745781: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-23 19:26:33.922455: W

In [2]:
%reload_ext autoreload
%autoreload 2

In [ ]:
class FullImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = Path(root_dir)
        self.transform = transform

        self.files = [x for x in self.root_dir.glob("**/*") if x.is_file()]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx: int):
        img_path = self.files[idx]

        image = Image.open(img_path).convert("RGB")
        width = image.width
        height = image.height

        if self.transform is not None:
            image = (self.transform(image),)

        return {"image": image, "path": img_path, "width": width, "height": height}


transform = transforms.Compose(
    [
        transforms.Resize((800, 800)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ],
)


def collate_batch(batch):
    images = torch.stack([elem["image"] for elem in batch])
    paths = [elem["path"] for elem in batch]
    widths = [elem["width"] for elem in batch]
    heights = [elem["height"] for elem in batch]
    return {"images": images, "paths": paths, "widths": widths, "heights": heights}


dataset = FullImageDataset("bird_dataset/train_images", transform=transform)
dataloader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    collate_fn=collate_batch,
)


In [ ]:
model = (torch.hub.load("facebookresearch/detr", "detr_resnet50", pretrained=True),)
model.eval()


In [ ]:
bird_index = CLASSES.index("bird")


def box_cxcywh_to_square_xyxy(x, width, height):
    x_c, y_c, w, h = x
    h = max(h, w * width / height)
    w = max(w, h * height / width)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h), (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.FloatTensor(b)


def rescale_bboxes(out_bbox, width, height):
    b = box_cxcywh_to_square_xyxy(out_bbox, width, height)
    b = b * torch.tensor([width, height, width, height], dtype=torch.float32)
    return b


with tqdm(dataloader, unit="batch") as vepoch:
    model.eval()
    for batch in vepoch:
        images = batch["images"]
        paths = batch["paths"]
        widths = batch["widths"]
        heights = batch["heights"]

        outputs = model(images)
        for pred_boxes, pred_logits, path, width, height in zip(
            outputs["pred_boxes"], outputs["pred_logits"], paths, widths, heights
        ):
            probas = pred_logits.softmax(-1)[:-1]
            keep = probas[bird_index] > 0.7

            im = Image.open(path).convert("RGB")
            parts = list(path.parts)
            parts[1] += "_cropped"
            cropped_path = Path(*parts)
            cropped_path.parents[0].mkdir(parents=True, exist_ok=True)
            if torch.sum(keep) > 0:
                largest_box_index = np.argmin(
                    [(box[2] * box[3]).item() for box in pred_boxes[keep]]
                )
                bboxes_scaled = rescale_bboxes(
                    pred_boxes[keep][largest_box_index], width, height
                )
                im = im.crop(bboxes_scaled.tolist())
            else:
                print(f"Image at {path} not cropped")
            im.save(cropped_path)


Images with problem on train:

* `004.Groove_billed_Ani/Groove_Billed_Ani_0068_1538.jpg`
* `030.Fish_Crow/Fish_Crow_0024_26064.jpg`
* `028.Brown_Creeper/Brown_Creeper_0111_24590.jpg`

Images with problem on validation:

* `026.Bronzed_Cowbird/Bronzed_Cowbird_0002_796244.jpg`

Image with problem on test:

* `mistery_category/0247efd7b9d47d036bb4390202a13e69.jpg`
* `mistery_category/8ede0bc5a4976385dcfe6e38feaf90c2.jpg`
* `mistery_category/15d6c5d42688cbc390e9ba241e93b941.jpg`
* `mistery_category/0ab685b1515b7d4c76691e8373a65f47.jpg`
* `mistery_category/4836bedeec2a2617ee33922ebf89995a.jpg`
* `mistery_category/8367922ad8b74c6047ea82f4d527ce04.jpg`